In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import json,re
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum,get_min_max,compute_average
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from joblib import Parallel, delayed

#TODO polars



df_train=pd.read_csv('dataset/origen.csv')
genre_types = df_train['genres_x'].unique().tolist()

genre_types = [genre.split(",") if type(genre)==str else ['Missing'] for genre in genre_types]
genre_types = [item for sublist in genre_types for item in sublist]
genre_types = list(set(genre_types))



status_types = df_train['status'].unique().tolist()
status_types.remove(np.nan)
status_types.append('Missing')
status_types.append('Canceled')
""" 
for i, status in enumerate(status_types):
    status_types[i] = f"status_{status}" """
#take a 10% sample out of train
df_test =  df_train.sample(frac=0.1, random_state=43)
#drop the sample from train
df_train.drop(df_test.index, inplace=True)

X_test = df_test.drop(["averageRating",'Unnamed: 0'], axis=1)
X = df_train.drop(["averageRating",'Unnamed: 0'], axis=1)
y = df_train["averageRating"]
y_test = df_test['averageRating']

del df_test,df_train



In [12]:

print(type(eval(X['production_companies'].iloc[15])[0]))


<class 'dict'>


In [13]:
X['runtimeMinutes'].replace(0, np.nan, inplace=True)
X['budget'].replace(0, np.nan, inplace=True)
X['revenue'].replace(0, np.nan, inplace=True)
X['titleType'].replace(0, np.nan, inplace=True)




In [14]:
print(X.production_countries.unique())

[nan "[{'iso_3166_1': 'IT', 'name': 'Italy'}]" '[]' ...
 "[{'iso_3166_1': 'FI', 'name': 'Finland'}, {'iso_3166_1': 'EE', 'name': 'Estonia'}, {'iso_3166_1': 'KE', 'name': 'Kenya'}]"
 "[{'iso_3166_1': 'IN', 'name': 'India'}, {'iso_3166_1': 'SE', 'name': 'Sweden'}, {'iso_3166_1': 'RS', 'name': 'Serbia'}, {'iso_3166_1': 'GR', 'name': 'Greece'}]"
 "[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'iso_3166_1': 'FR', 'name': 'France'}, {'iso_3166_1': 'PT', 'name': 'Portugal'}]"]


In [15]:

categorical_columns_with_nan = [
    c for c in X.columns if X[c].dtype == 'O' and X[c].isnull().any()]

categorical_columns_without_nan = [
    c for c in X.columns if X[c].dtype == 'O' and X[c].notnull().all()]



numerical_columns_with_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].isnull().any()]
numerical_columns_without_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].notnull().all()]

In [16]:
class TransformFromDict(BaseEstimator, TransformerMixin):
    #Voy a intentar explicar esto lo mejor que puedas
    def __init__(self, variables:list):
        #
        if type(variables) == str:
            variables = [variables]     
        self.variables = variables

    
    def fit(self, X, y=None):
        return self
    
    
    def transform(self, X):
        def get_str(x):
            #x is missing
            if x is np.nan or x is None or x == 'Missing' or x == 'nan' or x == '[]' or x == '':
                return ''
            x = eval(x)
            names = []
            if type(x)== list:
                for item in x:
                    if type(item)==dict:
                        name = item.get('name', '')
                    else:
                        name = ''
                    names.append(name)
            else:
                names.append('')
            return ','.join(names)
        for group in self.variables:
            X[group] = X[group].apply(get_str)
        return X
lista = []


In [17]:
from feature_engine.imputation import AddMissingIndicator, CategoricalImputer, MeanMedianImputer
from feature_engine.encoding import OneHotEncoder,RareLabelEncoder
from feature_engine.selection import  DropFeatures
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
def interpolate_numericals(df, group, category, method='linear'):
    # Fill in missing values using interpolation
    df[category].replace(0, np.nan, inplace=True)
    df[category].interpolate(method=method, inplace=True)
    
    # Store the fitted values in a dictionary
    if group not in interpolate_numericals.fitted_values:
        interpolate_numericals.fitted_values[group] = {}
    interpolate_numericals.fitted_values[group][category] = df[category].copy()
    
    return df

# Initialize the fitted values dictionary
interpolate_numericals.fitted_values = {}
for to_remove in ['budget','revenue']:
    try:
        numerical_columns_with_nan.remove(to_remove)
    except ValueError:
        pass

def check(x):
    print(f"Number of missing values in titleType before categorical imputer: {x['titleType'].isna().sum()}")
    return x

pipeline_preprocessing = Pipeline([
    ('turnToNan', FunctionTransformer(lambda x: x.assign(runtimeMinutes=x['runtimeMinutes'].replace(0, np.nan),
                                                          budget=x['budget'].replace(0, np.nan),
                                                          revenue=x['revenue'].replace(0, np.nan)))),
    ('numericalMissingIndicator', AddMissingIndicator(variables=numerical_columns_with_nan)),
    ('categoricalMissingIndicator', AddMissingIndicator(variables=categorical_columns_with_nan)),
    ('runtimeTransform', FunctionTransformer(lambda x: interpolate_numericals(x, 'titleType', 'runtimeMinutes'))),
    ('budgetTransform', FunctionTransformer(lambda x: interpolate_numericals(x, 'titleType', 'budget'))),
    ('revenueTransform', FunctionTransformer(lambda x: interpolate_numericals(x, 'titleType', 'revenue'))),
    ('endYearsTransform', FunctionTransformer(lambda X: X.assign(endYear=X.apply(lambda x: x['startYear'] if x['endYear'] == 0 else x['endYear'], axis=1)), validate=False)),
    ('categoricalImputer', CategoricalImputer(variables=categorical_columns_with_nan,fill_value='Missing')), 
    ('meanMedianImputer', MeanMedianImputer(imputation_method='median', variables=numerical_columns_with_nan)),
    ('getSTRsFromDicts', TransformFromDict(variables=['production_companies','production_countries'])),
    #('tvCategories',RareLabelEncoder(tol=0.01,n_categories=6, variables=['titleType'],replace_with='Other_TV')),
])







In [18]:
class GetExpDict(BaseEstimator, TransformerMixin):
    #Voy a intentar explicar esto lo mejor que puedas
    def __init__(self, group:list, categories:list,targets:list=[''],simple_only:list = []):
        #
        if type(group) == str:
            group = [group]
        if type(categories) == str:
            categories = [categories]
        if type(targets) == str:
            targets = [targets]
        group.extend(simple_only)
        self.group = group
        self.categories = categories
        self.simple_only = simple_only
        self.targets = targets

    def process_group_category_target(self,group, category, target, X):
        #print('Processing group:',group, category, target)
        categorias = X[category].unique().tolist()
        categorias = [x.split(',') if type(x) == str else [] for x in categorias]
        categorias = [item for sublist in categorias for item in sublist]
        categorias = list(set(categorias))
        grupo = X.groupby(group).apply(lambda x: [(g, v) for g, v in zip(x[category], x[target] if target!= 'exp' else [1]*len(x))]).to_dict()
        if '0' in grupo:
            del grupo['0']
        return (group, grupo, categorias, target)


    def fit(self, X, y=None):
        print(self.group, self.categories, self.targets)
        print('creating parallel fitting job')
        results = Parallel(n_jobs=-1)(delayed(self.process_group_category_target)(group, category, target, X) for group in self.group for category in self.categories for target in self.targets)
        grupos = [result for result in results]
        print('done')
        self.diccionarios = Parallel(n_jobs=-1)(delayed(self._fit_group)(group, grupo, categorias,target) for group, grupo, categorias,target in grupos)
        print('done')
        return self
    
    def _fit_group(self, group, grupo, categorias, target):
        # Create a dictionary with directors as keys and their shows as values
        
        dict_limpio = {}
        for sub_grupo, shows in grupo.items():
            for dir in sub_grupo.split(','):
                dict_limpio[dir] = shows

        # Initialize a dictionary to count categories
        cat_counts = {}
        for sub_grupo in dict_limpio.keys():
            cat_counts[sub_grupo] = {}
            for cat in categorias:
                cat_counts[sub_grupo][cat] = [0, 0]

        # Update the count for each category
        for sub_grupo, values in dict_limpio.items():
            for v in values:
                cat = v[0]
                count = v[1]
                if cat in cat_counts[sub_grupo]:
                    cat_counts[sub_grupo][cat][0] += count
                    cat_counts[sub_grupo][cat][1] += 1

        # Compute the average for each category and store it in authors_xp
        authors_xp = {}
        for sub_grupo, cat_dict in cat_counts.items():
            authors_xp[sub_grupo] = {}
            for cat, counts in cat_dict.items():
                authors_xp[sub_grupo][cat] = compute_average(counts)
        
        return group, authors_xp, categorias, target
    
    def transform(self, X):
        # Define a helper function for parallel processing
        def process_group(group, diccionario, categorias, target):
            new_columns = {}
            if group not in self.simple_only:
                new_columns.update({
                    f'{group}_{target}_{category}': X[group].apply(sum_into_column, args=(diccionario, category,))
                    for category in categorias
                })
            new_columns.update({
                f'{group}_min': X[group].apply(get_min_max, args=(diccionario, 0, categorias,)),
                f'{group}_max': X[group].apply(get_min_max, args=(diccionario, 1, categorias,))
            })
            return new_columns

        # Run the helper function in parallel
        results = Parallel(n_jobs=-1)(delayed(process_group)(group, diccionario, categorias, target) for group, diccionario, categorias, target in self.diccionarios)

        # Combine the results
        new_columns = {k: v for result in results for k, v in result.items()}

        X = pd.concat([X, pd.DataFrame(new_columns)], axis=1)
        return X




pipeline_exp = Pipeline([
    ('set_exp_dict_genres', GetExpDict(group=['directors',"writers"], categories=['genres_x','titleType'],simple_only=['production_companies','production_countries'],targets=['numVotes','exp','runtimeMinutes'])),

])


In [19]:
def okay(x):
    print("Okay")
    return x
def binarize_genres(X):
    mlb = MultiLabelBinarizer()
    print('0')
    binarized_genres = mlb.fit_transform(X['genres_x'].str.split(','))
    df_binarized_genres = pd.DataFrame(binarized_genres, columns=mlb.classes_)
    
    print('1')
    for column in genre_types:
        if column not in df_binarized_genres.columns:
            df_binarized_genres[column] = 0
    print('2')
    df_binarized_genres = pd.concat([X, df_binarized_genres], axis=1)
    print('3')
    return df_binarized_genres

#TODO: Comparar a ver si es mejor ni usar order, season y episodeNumber

pipeline = Pipeline([('preprocessing',pipeline_preprocessing),
                     ('exp',pipeline_exp),
    ('okaycheck', FunctionTransformer(okay)),
    ('okaycheck01', FunctionTransformer(okay)),
    ('binarizeGenres', FunctionTransformer(binarize_genres)),
    ('okaycheck0', FunctionTransformer(okay)),
    ('removeEmptyRows', FunctionTransformer(lambda x: x.dropna(subset=['directors']))),
    ('okaycheck1', FunctionTransformer(okay)),
    
    ('oneHotEncoder', OneHotEncoder(variables=['titleType',"status"])),
    ('okaycheck2', FunctionTransformer(okay)),
    ('removeCategorical',FunctionTransformer(lambda x: x.drop(x.select_dtypes(include=['object']).columns, axis=1))), 
    ('okaycheck3', FunctionTransformer(okay)),

    ])

In [20]:
pipeline.fit(X,y) #1m 13.1

['directors', 'writers', 'production_companies', 'production_countries'] ['genres_x', 'titleType'] ['numVotes', 'exp', 'runtimeMinutes']
creating parallel fitting job
done
done
Okay
Okay
0
1
2
3
Okay
Okay
Okay


Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('turnToNan',
                                  FunctionTransformer(func=<function <lambda> at 0x000001CC660ADBC0>)),
                                 ('numericalMissingIndicator',
                                  AddMissingIndicator(variables=['runtimeMinutes',
                                                                 'seasonNumber',
                                                                 'episodeNumber',
                                                                 'ordering',
                                                                 'isOriginalTitle',
                                                                 'popularity',
                                                                 'runtime'])),
                                 ('categoricalMissingIndicator',
                                  AddMissingIndicator(variable...
                 FunctionTransformer(func=<function okay at 0x000001CC012625C0>)),
                ('oneHotEncoder',
                 OneHotEncoder(variables=['titleType', 'status'])),
                ('okaycheck2',
                 FunctionTransformer(func=<function okay at 0x000001CC012625C0>)),
                ('removeCategorical',
                 FunctionTransformer(func=<function <lambda> at 0x000001CC0119B1A0>)),
                ('okaycheck3',
                 FunctionTransformer(func=<function okay at 0x000001CC012625C0>))])

In [21]:
X_pred = pd.read_csv('dataset/testear.csv').drop(['Unnamed: 0'],axis=1)
jjjj = pipeline.transform(X)
#293 5m 42.8

Okay
Okay
0
1
2
3
Okay
Okay
Okay
Okay


In [ ]:
for column in jjjj.columns:
    print(column)

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor
# import cupy as cp
# import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split



In [23]:
regr = XGBRegressor(n_estimators=1100,eta=0.025,
                        max_depth=19,gamma = 0.20,
                        colsample_bytree = 0.7,colsample_bylevel=0.7,colsample_bynode=0.8,
                        tree_method = 'hist',
                        max_cached_hist_node=262144) # type: ignore
#regr = XGBRegressor(n_estimators=1200, eta=0.35, max_depth=7, multi_strategy="multi_output_tree", min_child_weight=1, subsample=1, colsample_bytree=1, gamma=0, alpha=0)
print("Training a XGBRegressor")
regr.fit(jjjj, y)
print("Finished training the XGBRegressor")
#Tarda 49 minutos con esos settings



Training a XGBRegressor
Finished training the XGBRegressor


In [41]:
#Grid search
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [1000, 1500, 2000],
    'max_depth': [6, 8, 10,17,19,20],
    'gamma':[0.1,0.2,0.3,0.5,0.7,1],
    'min_child_weight': [1,2,3,4],
}

regr = XGBRegressor()
grid_search = GridSearchCV(regr, param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(jjjj, y)
print(grid_search.best_params_)
print(grid_search.best_score_)
""" 
regr.fit(jjjj, y)
regr.score(X_test,y_test) #8 ~ 0.4165 # 5000 y 4 = 0.3958 """

KeyboardInterrupt: 

In [28]:

#X_test = pipeline.transform(X_test)
print("Predicting")
score = regr.score(X_test, y_test) #4797

print(f"R^2 score on testing data: {score:.4f}") 
# RMSE
from sklearn.metrics import mean_squared_error

y_pred = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
#print(set(X_pred.columns))

Predicting
R^2 score on testing data: 0.3928
RMSE: 1.095849


In [ ]:
X_pred = pipeline.transform(X_pred)
Yest = regr.predict(X_pred).clip(1, 10)
salida = pd.DataFrame(data={"averageRating": Yest})
salida.index = X_pred.index
salida.to_csv("predicciones/pred40.csv", sep=',',index=True,  index_label='Id')